In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
keyword_extraction = "/content/gdrive/MyDrive/CSE 6242 Project/Keyword Extraction"

In [3]:
path = keyword_extraction

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("hyperpartisan_news_detection", "bypublisher")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/150000 [00:00<?, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 600000
    })
    validation: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 150000
    })
})


In [8]:
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [9]:
#Data Preprocessing
print("\n👉 First item 'dataset[0]':")
print(dataset['train'][0])
print(len(dataset['train']))

error_values = ['null', 'n/a', 'nan', 'none', '']

# Filter out rows with errors
def clean_dataset(example):
    for column in example.keys():
        if isinstance(example["text"], str) and example["text"].lower() in error_values:
            return False
        if pd.isnull(example["bias"]):
            return False
        if pd.isnull(example["title"]):
            return False
    return True

cleaned_dataset = dataset.filter(clean_dataset)

# Output the cleaned dataset
print("Cleaned dataset size:", len(cleaned_dataset['train']))




👉 First item 'dataset[0]':
{'text': '<p>When explaining her decision to reevaluate Title IX guidelines as they pertain to sexual assault on college campuses, Secretary of Education Betsy DeVos <a href="https://www.nbcnews.com/news/us-news/betsy-devos-overhaul-obama-era-guidance-campus-sex-assault-n799471" type="external">said</a>: &#8220;Every survivor of sexual misconduct must be taken seriously. Every student accused of sexual misconduct must know that guilt is not predetermined.&#8221;</p> \n\n<p>The Obama administration&#8217;s changes to Title IX have been <a href="" type="internal">criticized</a> for, among other things, substantially lowering the burden of proof as it pertains to sexual assault, as well as denying elements of due process to the accused.</p> \n\n<p>However, many progressives are lashing out at DeVos because they hate her, and also rape culture and stuff.</p> \n\n<p>Perhaps the most grotesque attack came when Rob Ranco, a Texas attorney, tweeted Friday that &#822

Filter:   0%|          | 0/600000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/150000 [00:00<?, ? examples/s]

Cleaned dataset size: 599884


In [10]:
#cleaned dataset
# gensim ---> preprocessing, spacy ---> tokenization

!pip install gensim spacy pyate
!python -m spacy download en_core_web_sm


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.7 MB/s eta 0:00:00
  Created wheel for pyate: filename=pyate-0.5.5-py3-none-any.whl size=18268 sha256=d8d0a077710890af2ca338018e2570d64659fa62392e44a8e6b9fecb90eb3681
  Stored in directory: /root/.cache/pip/wheels/c7/03/03/02f61c63a425a29ab036a5f56cd0d7548484354963f4aabd01
Successfully built pyate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
print(cleaned_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 599884
    })
    validation: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 149235
    })
})


In [12]:
import spacy
from pyate import combo_basic
from collections import Counter

In [13]:
# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

In [14]:
def preprocess_text(text): # Preprocesses text by removing stopwords and punctuation.
    doc = nlp(text)
    clean_text = ' '.join(token.lemma_ for token in doc if not token.is_stop and not token.is_punct)
    return clean_text

In [15]:
print(cleaned_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 599884
    })
    validation: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'],
        num_rows: 149235
    })
})


In [16]:
'text' in cleaned_dataset['train'].column_names

True

In [17]:
# cleaned_dataset['preprocessed_text'] = [preprocess_text(text) for text in cleaned_dataset['train']['text']]

# clean the text for every entry in the dataset

In [18]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
!pip install nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import networkx as nx
from collections import Counter
from datasets import load_dataset
import html
import time



start_time = time.time()

def remove_html_tags(text):

    # Unescape HTML entities first
    # clean = html.unescape(text)

    # Compile the regular expression for HTML tags
    # tag_re = re.compile('<.*?>')

    # tag_re = tag_re.sub(r'<[^<>]*>', '').apply(html.unescape)

    # tag_re = re.compile('<.*?>')
    # tag_re = tag_re.sub(r'<[^<>]*>', '').replace(text, '')
    # text = html.unescape(text)

    # # Use sub() to replace the HTML tags with an empty string
    # return tag_re.sub('', text).replace('<[^<>]*>', text)

    clean_text = re.sub(r'<[^>]*>', '', text)
    return clean_text

def preprocess_text(text):
    # Remove HTML tags
    text = remove_html_tags(text)

    # Remove punctuation and special characters, and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text).lower()

    return text

def position_rank(tokens, window_size=5):
    graph = nx.Graph()
    for i, token in enumerate(tokens):
        for j in range(i + 1, min(i + window_size, len(tokens))):
            if not graph.has_edge(token, tokens[j]):
                graph.add_edge(token, tokens[j], weight=0)
            graph[token][tokens[j]]['weight'] += 1

    scores = nx.pagerank(graph)
    return scores

# Extract text from the entries
# texts = cleaned_dataset['train']['text']

texts = cleaned_dataset['train'][slice(0, 1000, None)]['text'] #extract text for 1 entry

# Preprocess each text
preprocessed_texts = [preprocess_text(text) for text in texts]

# Tokenize and remove stopwords
stop_words = set(stopwords.words('english'))
keywords_list = []
for text in preprocessed_texts:
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Perform keyword extraction using PositionRank
    keywords = position_rank(filtered_tokens)

    # Sort the keywords by score
    sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)

    num_keywords = 5 #top 5 keywords
    top_keywords = [keyword for keyword, score in sorted_keywords[:num_keywords]]

    keywords_list.append(top_keywords) # Append to keyword list

end_time = time.time()
elapsed_time = (end_time - start_time) / 1000
print("Runtime for 1 article:", elapsed_time, "seconds")

# Display the keywords for each entry
for i, keywords in enumerate(keywords_list):
    print(f"Keywords for entry {i+1}: {keywords}")



Runtime for 1 article: 0.012406352996826173 seconds
Keywords for entry 1: ['sexual', 'ranco', 'also', 'assault', 'attorney']
Keywords for entry 2: ['degree', 'martin', 'trayvon', 'university', 'science']
Keywords for entry 3: ['greek', 'university', 'texas', 'said', 'fraternity']
Keywords for entry 4: ['jewish', 'day', 'unity', 'acheinu', 'people']
Keywords for entry 5: ['trump', 'carrier', 'jobs', 'indiana', 'company']
Keywords for entry 6: ['balls', 'red', 'white', '22000', 'winning']
Keywords for entry 7: ['medicare', 'plans', 'advantage', 'cuts', 'percent']
Keywords for entry 8: ['slavery', 'said', 'homosexuality', '8212', 'denominations']
Keywords for entry 9: ['school', 'says', 'garden', 'students', 'food']
Keywords for entry 10: ['wildlife', 'removal', 'rattlesnake', 'said', 'area']
Keywords for entry 11: ['trade', 'accord', 'also', 'would', 'agreement']
Keywords for entry 12: ['trump', 'court', 'fisa', 'he8217s', 'trump8217s']
Keywords for entry 13: ['maduro', 'said', 'proteste

In [21]:
#write to CSV
import csv

# Define your CSV file name
output = 'keywords.csv'

cleaned_dataset['train']

# Open the CSV file for writing
with open(output, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Entry Index', 'Keywords', 'Title', 'Text', 'Bias'])

    # # Write the keywords for each entry
    # for i, keywords in enumerate(keywords_list):
    #     # Write the entry index and the keywords joined by a comma
    #     writer.writerow([f"Entry {i+1}", ', '.join(keywords)])

    #write to CSV

    for i, (entry, keywords) in enumerate(zip(cleaned_dataset['train'], keywords_list)):
        # Extract information from the entry
        index = i + 1
        keywords = ', '.join(keywords)
        title = entry['title']
        text = entry['text']
        bias = entry['bias']

        # Write the information to the CSV file
        writer.writerow([index, keywords, title, text, bias])



In [22]:
import os
os.getcwd()

'/content'

In [23]:
location = "/content/gdrive/MyDrive/CSE 6242 Project/Keyword Extraction"
os.path.join(location, 'keywords.csv')

'/content/gdrive/MyDrive/CSE 6242 Project/Keyword Extraction/keywords.csv'

In [24]:
os.getcwd()

'/content'

In [25]:
# keywords = extract_keywords_with_positionrank(cleaned_dataset['train']['text'])

In [26]:
# keywords = extract_keywords_with_positionrank(cleaned_dataset)
# for article, keys in keywords.items():
#     print("Article:", article[:30], "...", "\nKeywords:", keys, "\n")







In [27]:
# from transformers import AutoTokenizer

# model_checkpoint = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# def tokenize_function(examples):
#     return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# tokenized_datasets = cleaned_dataset.map(tokenize_function, batched=True)

# from transformers import AutoModelForSequenceClassification

# num_labels = 6  # number of topics to classify in
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [28]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch
# import requests
# from datasets import load_dataset

# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name)

# # Function to tokenize and process chunks of text
# def process_chunks(text_chunks):
#     tokenized_chunks = [tokenizer.encode(chunk, add_special_tokens=True, max_length=512, truncation=True) for chunk in text_chunks]
#     return tokenized_chunks

# # Function to process articles from URLs
# def process_articles_from_urls(urls):
#     for url in urls:
#         response = requests.get(url)
#         article_text = response.text

#         # Split article into chunks of 512 tokens
#         text_chunks = [article_text[i:i+512] for i in range(0, len(article_text), 512)]

#         # Process chunks
#         tokenized_chunks = process_chunks(text_chunks)

#         # Perform inference for each chunk
#         for tokens in tokenized_chunks:
#             tokens_tensor = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension
#             with torch.no_grad():
#                 outputs = model(tokens_tensor)
#                 logits = outputs.logits
#                 probabilities = torch.softmax(logits, dim=-1)
#                 predicted_class = torch.argmax(probabilities).item()
#                 print("Predicted class:", predicted_class)
#                 # You can handle the results as needed

# # Process articles in train and validation sets
# train_urls = tokenized_datasets['train']['url']

# # Process articles from train set
# process_articles_from_urls(train_urls)

